# Systematics Notebook
From results of the phi fitting.  Before running this notebook the output from `src/fit/run-fitter.py` is needed.  

In [1]:
import glob 
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import pickle
import sys
import time 

%matplotlib inline

plt.rc('font', family='serif')
plt.rc('font', size=18)

### Load Configurations
There are several files with different results for phi-distributions.  

In [2]:
database_files = glob.glob('database/fit/*.csv')
print('Found %d files in the database.' % len(database_files))

Found 25 files in the database.


In [3]:
def load_database_files(file_list):
    
    dataframe_store = {}
    for f in file_list:
        dataframe_store[f] = pd.read_csv(f)
        
    return dataframe_store

In [4]:
dataframe_store = load_database_files(database_files)

Finally, fit the nominal phi distributions.

In [5]:
nominal = pd.read_csv('database/fit/sys.csv')

### Systematic Uncertainties 

In [6]:
def database_filename_parser(file_name):
    file_name = file_name.split('variation_')[-1].strip('.csv')
    
    tokens = file_name.split('_')
    n_tokens = len(tokens)

    parameter_name = '_'.join(tokens[0:n_tokens-1])
    index = int(tokens[-1])
    
    return index, parameter_name

In [7]:
def build_parameter_variation_dict(path_to_db = 'database/fit/'):

    parameters = {}

    database_files = glob.glob(path_to_db + 'variation*.csv')
    
    for database_file in database_files:
        index, parameter = database_filename_parser(database_file)
        
        if parameter in parameters.keys():
            parameters[parameter][index] = pd.read_csv(database_file)
        else:
            parameters[parameter] = {}
            parameters[parameter][index] = pd.read_csv(database_file)
            
    return parameters

In [8]:
def load_systematic_sources_list(file_name):
    systematic_sources = pickle.load(open(file_name, 'rb'))
    
    reverse_dict = {}

    for key, value in systematic_sources.iteritems():
        reverse_dict[value] = key
    
    return reverse_dict

In [9]:
parameter_variation = build_parameter_variation_dict()

In [10]:
for key, value in parameter_variation.iteritems():
    print(key, value.keys())

('dist_dcr3', [1, -1])
('dist_dcr1', [1, -1])
('dist_vz', [1, -1])
('dist_cc', [0, -1])
('dist_ec_edep', [1, -1])
('dist_ecsf', [1, -1])
('p_mes', [0, -1])
('alpha', [1, -1])
('missing_mass', [1, -1])
('dist_ecu', [1, -1])
('dist_ecw', [1, -1])
('dist_ecv', [1, -1])


In [11]:
systematic_sources = load_systematic_sources_list('systematic_sources.pkl')

In [12]:
def add_systematics(nominal_fit, parameter_variation, systematic_sources):

    nominal_fit_sys = nominal_fit.copy(deep=True)
    nominal_fit_sys['sys_total_0'] = np.zeros(len(nominal_fit_sys))
    nominal_fit_sys['sys_total_1'] = np.zeros(len(nominal_fit_sys))
    nominal_fit_sys['sys_total_2'] = np.zeros(len(nominal_fit_sys))
    
    for key in parameter_variation.keys():

        min_index = parameter_variation[key].keys()[0]    
        max_index = parameter_variation[key].keys()[-1]

        merged_data = pd.merge(parameter_variation[key][min_index], 
                 parameter_variation[key][max_index],
                 on = ['axis', 'axis_bin'])

        merged_data[systematic_sources[key] + '_par0'] = np.abs(merged_data.par_0_y - merged_data.par_0_x)
        merged_data[systematic_sources[key] + '_par1'] = np.abs(merged_data.par_1_y - merged_data.par_1_x)
        merged_data[systematic_sources[key] + '_par2'] = np.abs(merged_data.par_2_y - merged_data.par_2_x)

        merge_cols = ['axis', 'axis_bin', systematic_sources[key] + '_par0', 
                     systematic_sources[key] + '_par1', systematic_sources[key] + '_par2']

        nominal_fit_sys = pd.merge(nominal_fit_sys, merged_data[merge_cols], 
                                   on = ['axis', 'axis_bin'])
    
        nominal_fit_sys.sys_total_0 += nominal_fit_sys[systematic_sources[key] + '_par0']**2
        nominal_fit_sys.sys_total_1 += nominal_fit_sys[systematic_sources[key] + '_par1']**2
        nominal_fit_sys.sys_total_2 += nominal_fit_sys[systematic_sources[key] + '_par2']**2


    nominal_fit_sys.sys_total_0 = np.sqrt(nominal_fit_sys.sys_total_0)
    nominal_fit_sys.sys_total_1 = np.sqrt(nominal_fit_sys.sys_total_1)
    nominal_fit_sys.sys_total_2 = np.sqrt(nominal_fit_sys.sys_total_2)

    
    return nominal_fit_sys

In [13]:
nominal_fit_sys = add_systematics(nominal, parameter_variation, systematic_sources)

In [14]:
nominal_fit_sys.to_csv('results/fit/sys.csv', index=False)

In [15]:
nominal_fit_sys.head(24)

,axis,axis_bin,axis_max,axis_min,err_0,err_1,err_2,par_0,par_1,par_2,...,sys_9_par2,sys_10_par0,sys_10_par1,sys_10_par2,sys_5_par0,sys_5_par1,sys_5_par2,sys_12_par0,sys_12_par1,sys_12_par2
0,pt,0,0.202108,0.000284,0.005352,0.412564,0.482686,0.023401,0.005927,-0.320898,...,0.020015,0.000983,0.081365,0.119970,0.001109,0.053782,0.069089,0.002105,0.015896,0.102437
1,pt,1,0.274544,0.202108,0.006552,0.417063,0.513516,0.026041,-0.442958,-0.008075,...,0.036247,0.003853,0.045680,0.190870,0.000623,0.055744,0.065528,0.000646,0.087085,0.083373
2,pt,2,0.328453,0.274544,0.007196,0.389611,0.398448,0.029176,0.154819,-0.401798,...,0.043711,0.001525,0.092949,0.011717,0.000281,0.123901,0.005453,0.001332,0.049205,0.090000
3,pt,3,0.376169,0.328453,0.004918,0.410559,0.432942,0.034254,-0.091160,-0.253055,...,0.073261,0.000327,0.013976,0.059410,0.001394,0.030079,0.026911,0.001182,0.067643,0.070051
4,pt,4,0.422123,0.376169,0.007351,0.358476,0.438142,0.035249,0.047944,-0.301508,...,0.019235,0.002919,0.083515,0.077375,0.002006,0.073412,0.003738,0.001288,0.044306,0.081577
5,pt,5,0.469541,0.422123,0.005403,0.441346,0.474384,0.031860,-0.278000,-0.213023,...,0.002011,0.001021,0.163986,0.085947,0.000151,0.029757,0.053024,0.000993,0.054224,0.039969
6,pt,6,0.521940,0.469541,0.007531,0.450196,0.456658,0.033941,0.211874,-0.274106,...,0.075058,0.002117,0.076574,0.046669,0.000094,0.004044,0.014771,0.001196,0.150904,0.051431
7,pt,7,0.585257,0.521940,0.007250,0.417070,0.437486,0.034011,-0.153006,-0.327447,...,0.092999,0.002238,0.034590,0.023159,0.000259,0.034251,0.027875,0.002628,0.021549,0.053654
8,pt,8,0.674587,0.585257,0.006729,0.379462,0.502653,0.030805,-0.258803,-0.182275,...,0.029054,0.004765,0.015516,0.111407,0.002811,0.027486,0.018982,0.000511,0.087423,0.116740
9,pt,9,1.193084,0.674587,0.004521,0.433228,0.535124,0.019396,-0.415717,-0.069108,...,0.042478,0.000285,0.042747,0.035835,0.000089,0.061405,0.069257,0.001063,0.010211,0.105273
